In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df=pd.read_csv('../dataset/train.csv')

In [4]:
train_df.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [5]:
for i in range(len(train_df['image_id'])):
    train_df['image_id'][i]=train_df['image_id'][i]+'.jpg'

In [6]:
len(train_df['image_id'])

5508

In [7]:
train_df.head()

,image_id,breed
0,a8b3ad1dde.jpg,nerodia-erythrogaster
1,8b492b973d.jpg,pantherophis-vulpinus
2,929b99ea92.jpg,thamnophis-sirtalis
3,bbac7385e2.jpg,pantherophis-obsoletus
4,ef776b1488.jpg,agkistrodon-contortrix


In [8]:
train_df.tail()

,image_id,breed
5503,5f0c970adf.jpg,lampropeltis-triangulum
5504,f6a4287fac.jpg,pantherophis-alleghaniensis
5505,d3c10e63e1.jpg,thamnophis-sirtalis
5506,61ef50879c.jpg,pituophis-catenifer
5507,4791d8bf9a.jpg,lampropeltis-triangulum


In [9]:
from keras.preprocessing import image

Using TensorFlow backend.


In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=25,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

test_datagen = ImageDataGenerator()

In [17]:
train_ratio = 0.8
train_len = int(train_ratio * len(train_df))

In [19]:
train_generator=train_datagen.flow_from_dataframe(dataframe=train_df[0:train_len],
                                                directory="../dataset/train/",
                                                x_col="image_id",
                                                y_col="breed",
                                                subset="training",
                                                batch_size=128,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=(224, 224))

test_generator=test_datagen.flow_from_dataframe(dataframe=train_df[train_len:],
                                                directory="../dataset/train/",
                                                x_col="image_id",
                                                y_col="breed",
                                                subset="training",
                                                batch_size=128,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=(224, 224))

Found 4406 validated image filenames belonging to 35 classes.
Found 1102 validated image filenames belonging to 35 classes.


In [25]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Dropout, Dense, Flatten, AveragePooling2D

In [28]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

AttributeError: 'str' object has no attribute 'decode'

In [29]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

NameError: name 'baseModel' is not defined